In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

import os

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [16]:
sentence_length = 110

run_experiments = os.getenv("UVL_BERT_RUN_EXPERIMENTS","True") == "True"
pin_commits = os.getenv("UVL_BERT_PIN_COMMITS","True") == "True"

print(f"{run_experiments=}")
print(f"{pin_commits=}")

run_experiments=True
pin_commits=True


In [17]:
from tooling.config import Experiment, Transformation

base_experiment_config = Experiment(
    name="Base Config", iterations=1, force=False
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Domain_Level",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Interaction_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
)

In [18]:
from tooling.transformation import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)
hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

02:41:49 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Domain_Level': 1, 'Interaction_Level': 2, 'System_Level': 3}
02:41:49 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Task': 1, 'Domain_Data': 2, 'Activity': 3, 'Stakeholder': 4, 'System_Function': 5, 'Interaction': 6, 'Interaction_Data': 7, 'Workspace': 8, 'System_Level': 9}


## Train BiLSTM First Stage Model

In [19]:
from tooling.observability import get_run_id
from tooling.config import BiLSTMConfig, BiLSTM

from copy import deepcopy

bilstm_experiment_config = deepcopy(base_experiment_config)
bilstm_experiment_config.name = "Production: BiLSTM Train for Staged Application"

bilstm_config = BiLSTM(sentence_length=sentence_length)

bilstm_cfg: BiLSTMConfig = OmegaConf.structured(
    BiLSTMConfig(
        bilstm=bilstm_config,
        experiment=bilstm_experiment_config,
        transformation=levels_transformation_config,
    )
)

#bilstm_cfg.experiment.force = True

if run_experiments:
    from experiments.bilstm import bilstm
    bilstm(bilstm_cfg)


bilstm_run_id = get_run_id(bilstm_cfg, pin_commit = pin_commits)

print(bilstm_run_id)

print(mlflow.get_artifact_uri())

bilstm_run = mlflow.get_run(bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bilstm")

02:41:52 INFO:
bilstm:
  type: BiLSTM
  sentence_length: 110
  batch_size: 32
  number_epochs: 4
  verbose: 1
  weighted_classes: false
  learning_rate: 0.0001
experiment:
  name: 'Production: BiLSTM Train for Staged Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: true
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

02:41:52 INFO:New experiment. Running
02:41:52 INFO:Entering mlflow context
02:41:53 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
02:41:53 

Epoch 1/4
34/34 [==============================] - 6s 180ms/step - loss: 1.2503 - precision: 0.3333 - recall: 0.3563 - val_loss: 1.1118 - val_precision: 0.3172 - val_recall: 0.3333
Epoch 2/4
34/34 [==============================] - 6s 177ms/step - loss: 0.8755 - precision: 0.3172 - recall: 0.3333 - val_loss: 0.3759 - val_precision: 0.3172 - val_recall: 0.3333
Epoch 3/4
34/34 [==============================] - 6s 173ms/step - loss: 0.2107 - precision: 0.3172 - recall: 0.3333 - val_loss: 0.1725 - val_precision: 0.3172 - val_recall: 0.3333
Epoch 4/4
34/34 [==============================] - 6s 180ms/step - loss: 0.1635 - precision: 0.3172 - recall: 0.3333 - val_loss: 0.1553 - val_precision: 0.3172 - val_recall: 0.3333


02:43:10 INFO:Assets written to: /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/temp/bilstm/dazzling-snipe-142/0_model/assets
2023-08-15 14:43:19.442699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-15 14:43:19.558348: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-15 14:43:19.571388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-15 14:43:19.623863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-08-15 14:43:19.635237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 35ms/step


02:43:20 INFO:Logged iteration result res.precision=0.18282600526118 res.recall=0.25
02:43:20 INFO:Finished iteration=0
02:43:20 INFO:Breaking early after iteration=0 of 5 folds
02:43:21 INFO:Logged experiment result res.mean_precision=0.18282600526118 res.mean_recall=0.25
02:43:21 INFO:Left mlflow context


b97f80571648461688b5399a230d499a
mlflow-artifacts:/32/0ab4fdfb4c2b482ca797b592b6c9da01/artifacts


PosixPath('src/service/models/bilstm')

## Train SNER First Stage Model

In [7]:
from tooling.observability import get_run_id
from tooling.config import SNERConfig, SNER

from copy import deepcopy

sner_experiment_config = deepcopy(base_experiment_config)
sner_experiment_config.name = "Production: SNER Train for Staged Application"

sner_config = SNER()

sner_cfg = OmegaConf.structured(
    SNERConfig(
        sner=sner_config,
        experiment=sner_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.sner import sner
    sner(OmegaConf.create(sner_cfg))

sner_run_id = get_run_id(sner_cfg, pin_commit = pin_commits)
print(mlflow.get_artifact_uri())
print(sner_run_id)

sner_run = mlflow.get_run(sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{sner_run.info.artifact_uri}/0_model.ser.gz",
    dst_path=Path("./src/service/models/"),
)
try:
    Path("./src/service/models/sner.ser.gz").unlink()
except FileNotFoundError:
    pass
Path("./src/service/models/0_model.ser.gz").rename(
    "./src/service/models/sner.ser.gz"
)

11:53:16 INFO:
sner:
  type: SNER
experiment:
  name: 'Production: SNER Train for Staged Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

11:53:16 WARNING:Experiment was already run, aborting


mlflow-artifacts:/32/55df6bc6c58548069fd69f6146d4db82/artifacts
9b40a85a79a34363866cde507f5ea4f8


PosixPath('src/service/models/sner.ser.gz')

## Train BERT First Stage Model

In [8]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_1_experiment_config = deepcopy(base_experiment_config)
bert_1_experiment_config.name = "Production: BERT_1 Train for Staged Application"

bert_1_config = BERT(max_len=sentence_length)

bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert
    bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg, pin_commit = pin_commits)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")

11:53:18 INFO:Created a temporary directory at /var/folders/82/x_tg3xgx1px781gb8v4bny1r0000gn/T/tmp42bfdsdz
11:53:18 INFO:Writing /var/folders/82/x_tg3xgx1px781gb8v4bny1r0000gn/T/tmp42bfdsdz/_remote_module_non_scriptable.py
11:53:18 INFO:
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: 'Production: BERT_1 Train for Staged Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Interaction_Level
  workspace: Interaction_Level
  so

2ad9ba334cb04a70a3129e6309d2be0a


PosixPath('src/service/models/bert_1')

## Train BERT Second Stage Model for BERT First Stage

In [9]:
run_experiments

True

In [10]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bert_experiment_config = deepcopy(base_experiment_config)
bert_2_bert_experiment_config.name = "Production: BERT_2 Train for Staged Application"

bert_2_bert_config = StagedBERT(max_len=sentence_length)

bert_2_bert_cfg: DualModelStagedBERTConfig = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bert_config,
        experiment=bert_2_bert_experiment_config,
        transformation=label_transformation_config,
        first_model_bert=bert_1_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert
    dual_stage_bert(OmegaConf.create(bert_2_bert_cfg))

bert_2_bert_run_id = get_run_id(bert_2_bert_cfg, pin_commit = pin_commits)

print(bert_2_bert_run_id)

bert_2_bert_run = mlflow.get_run(bert_2_bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bert_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_2_bert")

11:53:38 INFO:
first_model_bert:
  bert:
    model: bert-base-uncased
    type: BERT
    max_len: 110
    train_batch_size: 32
    validation_batch_size: 32
    number_epochs: 5
    learning_rate_bert: 2.0e-05
    learning_rate_classifier: 0.01
    weight_decay: 0.01
    weighted_classes: false
  experiment:
    name: 'Production: BERT_1 Train for Staged Application'
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
first_model_bilstm: null
first_model_sner: null
ber

Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

11:54:09 INFO:Loading Model
11:54:09 INFO:Using device: mps
11:54:10 INFO:Creating hint column


Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

...

11:57:24 INFO:Logged iteration result res.precision=0.703754153738194 res.recall=0.6551194365013446


{'eval_loss': 0.360973060131073, 'eval_step': 8, 'eval_precision': 0.703754153738194, 'eval_recall': 0.6551194365013446, 'eval_label_count': 10, 'eval_runtime': 2.3081, 'eval_samples_per_second': 116.546, 'eval_steps_per_second': 3.899, 'epoch': 5.0}
{'train_runtime': 152.3414, 'train_samples_per_second': 35.283, 'train_steps_per_second': 1.116, 'train_loss': 0.312238670797909, 'epoch': 5.0}


11:57:30 INFO:Logged iteration result res.precision=0.7264684253132685 res.recall=0.6805139460710684
11:57:30 INFO:Logged iteration result res.precision=0.7264684253132685 res.recall=0.6805139460710684
11:57:30 INFO:Finished iteration=0
11:57:30 INFO:Logging model artifact (might take a while)
11:59:08 INFO:Breaking early after iteration=0 of 5 folds
11:59:09 INFO:Logged experiment result res.mean_precision=0.7264684253132685 res.mean_recall=0.6805139460710684
11:59:09 INFO:Left mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
a574cd93fe0b456dadaaac30e56e0aa5


PosixPath('src/service/models/bert_2_bert')

## Train BERT Second Stage Model for SNER First Stage

In [11]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_sner_experiment_config = deepcopy(base_experiment_config)
bert_2_sner_experiment_config.name = "Production: BERT_2 Train for Staged Application"

bert_2_sner_config = StagedBERT(max_len=sentence_length)

bert_2_sner_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_sner_config,
        experiment=bert_2_sner_experiment_config,
        transformation=label_transformation_config,
        first_model_sner=sner_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert
    dual_stage_bert(OmegaConf.create(bert_2_sner_cfg))

bert_2_sner_run_id = get_run_id(bert_2_sner_cfg, pin_commit = pin_commits)

print(bert_2_sner_run_id)

bert_2_sner_run = mlflow.get_run(bert_2_sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_sner_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_sner"))
except FileNotFoundError:
    pass

Path("./src/service/models/0_model").rename("./src/service/models/bert_2_sner")

11:59:28 INFO:
first_model_bert: null
first_model_bilstm: null
first_model_sner:
  sner:
    type: SNER
  experiment:
    name: 'Production: SNER Train for Staged Application'
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
  la

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


11:59:29 INFO:Using device: mps
11:59:30 INFO:Found existing run with run_id: 9b40a85a79a34363866cde507f5ea4f8 matching the configuration
11:59:30 INFO:Downloading run model from mlflow-artifacts:/33/9b40a85a79a34363866cde507f5ea4f8/artifacts/0_model.ser.gz


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


11:59:30 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
11:59:31 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
11:59:31 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67_100.json
11:59:33 INFO:Dataset Labels: transformed_dataset['labels']=['Activity', 'System_Level', '0', 'Stakeholder', 'Interaction_Data', 'Task', 'Domain_Data', 'System_Function', 'Interaction', 'Workspace']
11:59:33 INFO:Class weights: {'0': 1.0, 'Task': 1.0, 'Domain_Data': 1.0, 'Activity': 1.0, 'Stakeholder': 1.0, 'System_Function': 1.0, 'Interaction': 1.0, 'Interaction_Data': 1.0, 'Workspace': 1.0, 'System_Level': 1.0}
11:59:33 INFO:Configured maximal token sequence length: max_len = 110
11:59:34 INFO:Created fold datasets for fold: 0, store

...

12:02:06 INFO:Logged iteration result res.precision=0.6876090337775963 res.recall=0.632028136051705


{'eval_loss': 0.36508819460868835, 'eval_step': 8, 'eval_precision': 0.6876090337775963, 'eval_recall': 0.632028136051705, 'eval_label_count': 10, 'eval_runtime': 2.2024, 'eval_samples_per_second': 122.139, 'eval_steps_per_second': 4.086, 'epoch': 5.0}
{'train_runtime': 149.9144, 'train_samples_per_second': 35.854, 'train_steps_per_second': 1.134, 'train_loss': 0.24499363618738512, 'epoch': 5.0}


12:02:12 INFO:Logged iteration result res.precision=0.7207249044272385 res.recall=0.6588470895526491
12:02:12 INFO:Logged iteration result res.precision=0.7207249044272385 res.recall=0.6588470895526491
12:02:12 INFO:Finished iteration=0
12:02:12 INFO:Logging model artifact (might take a while)
12:03:49 INFO:Breaking early after iteration=0 of 5 folds
12:03:50 INFO:Logged experiment result res.mean_precision=0.7207249044272385 res.mean_recall=0.6588470895526491
12:03:50 INFO:Left mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
3385dbbaafaf4c589f0a7868a59d24ee


PosixPath('src/service/models/bert_2_sner')

## Train BERT Second Stage Model for BILSTM First Stage

In [12]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bilstm_experiment_config = deepcopy(base_experiment_config)
bert_2_bilstm_experiment_config.name = "Production: BERT_2 Train for Staged Application"

bert_2_bilstm_config = StagedBERT(max_len=sentence_length)

bert_2_bilstm_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bilstm_config,
        experiment=bert_2_bilstm_experiment_config,
        transformation=label_transformation_config,
        first_model_bilstm=bilstm_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert
    dual_stage_bert(OmegaConf.create(bert_2_bilstm_cfg))

bert_2_bilstm_run_id = get_run_id(bert_2_bilstm_cfg, pin_commit = pin_commits)

print(bert_2_bilstm_run_id)

bert_2_bilstm_run = mlflow.get_run(bert_2_bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename(
    "./src/service/models/bert_2_bilstm"
)

12:04:09 INFO:
first_model_bert: null
first_model_bilstm:
  bilstm:
    type: BiLSTM
    sentence_length: 110
    batch_size: 32
    number_epochs: 32
    verbose: 1
    weighted_classes: false
    learning_rate: 0.0001
  experiment:
    name: 'Production: BiLSTM Train for Staged Application'
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: prolific
    lower_case: false
    force: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Interaction_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System_Level
first_model_sner: null
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_s

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


12:04:09 INFO:Using device: mps
12:04:10 INFO:Found existing run with run_id: 699064dbd3854a86a6e0f106e153739e matching the configuration
12:04:10 INFO:Downloading run model from mlflow-artifacts:/32/699064dbd3854a86a6e0f106e153739e/artifacts/0_model


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


12:04:12 INFO:loading projection weights from /Users/bockstaller/gensim-data/glove-twitter-100/glove-twitter-100.gz
12:04:47 INFO:KeyedVectors lifecycle event {'msg': 'loaded (1193514, 100) matrix of type float32 from /Users/bockstaller/gensim-data/glove-twitter-100/glove-twitter-100.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-08-15T12:04:47.320114', 'gensim': '4.3.1', 'python': '3.11.4 (main, Jun 20 2023, 19:14:10) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'load_word2vec_format'}
12:04:47 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
12:04:47 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
12:04:48 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67

...

12:07:50 INFO:Logged iteration result res.precision=0.7020240090507268 res.recall=0.6558736978600617


{'eval_loss': 0.35905617475509644, 'eval_step': 8, 'eval_precision': 0.7020240090507268, 'eval_recall': 0.6558736978600617, 'eval_label_count': 10, 'eval_runtime': 2.2789, 'eval_samples_per_second': 118.04, 'eval_steps_per_second': 3.949, 'epoch': 5.0}
{'train_runtime': 152.9563, 'train_samples_per_second': 35.141, 'train_steps_per_second': 1.111, 'train_loss': 0.31347095265107994, 'epoch': 5.0}


12:07:55 INFO:Logged iteration result res.precision=0.7211404156400778 res.recall=0.6836553227628458
12:07:56 INFO:Logged iteration result res.precision=0.7211404156400778 res.recall=0.6836553227628458
12:07:56 INFO:Finished iteration=0
12:07:56 INFO:Logging model artifact (might take a while)
12:09:34 INFO:Breaking early after iteration=0 of 5 folds
12:09:35 INFO:Logged experiment result res.mean_precision=0.7211404156400778 res.mean_recall=0.6836553227628458
12:09:35 INFO:Left mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
d54d1caaff404a2b8cbbfa9c479dabad


PosixPath('src/service/models/bert_2_bilstm')

## Train BERT E2E Model

In [13]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_experiment_config = deepcopy(base_experiment_config)
bert_experiment_config.name = "Production: BERT Train for E2E Application"

bert_config = BERT(max_len=sentence_length)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert
    bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg, pin_commit = pin_commits)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")

12:09:54 INFO:
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 110
  train_batch_size: 32
  validation_batch_size: 32
  number_epochs: 5
  learning_rate_bert: 2.0e-05
  learning_rate_classifier: 0.01
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: 'Production: BERT Train for E2E Application'
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: prolific
  lower_case: false
  force: false
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level

12:09:55 INFO:New experiment. Running
12:09:55 INFO:Entering mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


12:09:55 INFO:Using device: mps
12:09:55 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_1_33.json
12:09:55 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_34_66.json
12:09:56 INFO:Importing dataset: prolific from /Users/bockstaller/code/uvl-tore-classifier-bert/src/data/datasets/prolific/TORE_Coded_Answers_67_100.json
12:09:58 INFO:Dataset Labels: transformed_dataset['labels']=['Activity', 'System_Level', '0', 'Stakeholder', 'Interaction_Data', 'Task', 'Domain_Data', 'System_Function', 'Interaction', 'Workspace']
12:09:58 INFO:Class weights: {'0': 1.0, 'Task': 1.0, 'Domain_Data': 1.0, 'Activity': 1.0, 'Stakeholder': 1.0, 'System_Function': 1.0, 'Interaction': 1.0, 'Interaction_Data': 1.0, 'Workspace': 1.0, 'System_Level': 1.0}
12:09:58 INFO:Configured maximal token sequence length: max_len = 110
12:10:00 INFO:Created 

Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

...

12:12:31 INFO:Logged iteration result res.precision=0.6987279850930139 res.recall=0.6547137195652447


{'eval_loss': 0.3600654602050781, 'eval_step': 8, 'eval_precision': 0.6987279850930139, 'eval_recall': 0.6547137195652447, 'eval_label_count': 10, 'eval_runtime': 2.1877, 'eval_samples_per_second': 122.961, 'eval_steps_per_second': 4.114, 'epoch': 5.0}
{'train_runtime': 151.3343, 'train_samples_per_second': 35.517, 'train_steps_per_second': 1.123, 'train_loss': 0.32144239089068244, 'epoch': 5.0}


12:12:37 INFO:Logged iteration result res.precision=0.7178643892293535 res.recall=0.6810540380395174
12:12:37 INFO:Logged iteration result res.precision=0.7178643892293535 res.recall=0.6810540380395174
12:12:37 INFO:Finished iteration=0
12:12:37 INFO:Logging model artifact (might take a while)
12:14:15 INFO:Breaking early after iteration=0 of 5 folds
12:14:16 INFO:Logged experiment result res.mean_precision=0.7178643892293535 res.mean_recall=0.6810540380395174
12:14:16 INFO:Left mlflow context


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
c73f5384ee3642edbb361043cbb3f646


PosixPath('src/service/models/bert')